### PET-CT 医学影像数据可视化

- 参数设置

In [ ]:
ct_path = "data/CT_PSMA_60Min_Patient001.nii.gz"
pet_path = "data/PET_PSMA_60Min_Patient001.nii.gz"

- 函数定义

In [ ]:
import SimpleITK as sitk
from scipy.ndimage import zoom
import numpy as np

# Load images and get metadata of Size, Origin, Spacing
def get_image_info(path):
    img = sitk.ReadImage(path)
    return img, {
        "shape": img.GetSize(),
        "origin": img.GetOrigin(),
        "spacing": img.GetSpacing(),
    }

# from Size, Origin, Spacing calculate the end origin
def calculate_end_origin(info):
    size = info["shape"]
    origin = info["origin"]
    spacing = info["spacing"]
    end_origin = (
        origin[0] + spacing[0] * (size[0] - 1),
        origin[1] + spacing[1] * (size[1] - 1),
        origin[2] + spacing[2] * (size[2] - 1),
    )
    return end_origin

# set isotropy of 3D images if needed
def set_isotropy(arr, spacing, new_spacing=1.0):
    zoom_factors = [s / new_spacing  for s in spacing]
    return zoom(arr, zoom_factors, order=1)

# Normalize image to [0, 1] range using 1st and 99th percentiles to avoid outliers.
def normalize(img, eps=1e-8):
    vmin, vmax = np.percentile(img, (1, 99))
    img = np.clip(img, vmin, vmax)
    return (img - vmin) / (vmax - vmin + eps)

- 了解医学影像空间位置

In [ ]:
# Get image infos
ct_img, ct_info = get_image_info(ct_path)
pet_img, pet_info = get_image_info(pet_path)
# Calculate end origins
ct_end_origin = calculate_end_origin(ct_info)
pet_end_origin = calculate_end_origin(pet_info)

print(
    f"CT shape: {ct_info['shape']}\n"
    f"CT spacing: {ct_info['spacing']}\n"
    f"CT start origin: {ct_info['origin']}\n"
    f"CT end origin: {ct_end_origin}\n\n"
    f"PET shape: {pet_info['shape']}\n"
    f"PET spacing: {pet_info['spacing']}\n"
    f"PET start origin: {pet_info['origin']}\n"
    f"PET end origin: {pet_end_origin}"
)

- PET 影像重采样到 CT 的空间坐标系

In [ ]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

# 把 PET 影像重采样到 CT 的空间坐标系里
# Resample PET into CT space using CT as reference (origin, spacing, direction, size).
pet_resampled = sitk.Resample(
    pet_img,                    # Resample the PET image
    ct_img,                     # Use CT image as reference
    sitk.Transform(),           # No additional transform
    sitk.sitkLinear,            # Linear interpolation
    0.0,                        # Default pixel value for areas outside the original PET image
    pet_img.GetPixelID(),       # Use the same pixel type as the original PET image
)

ct_arr = sitk.GetArrayFromImage(ct_img)
pet_arr = sitk.GetArrayFromImage(pet_resampled)
print(f"Resample PET into CT space using CT as reference: \nCT {ct_arr.shape} vs PET {pet_arr.shape}")

pet_ct_info = {
    "shape": pet_resampled.GetSize(),
    "origin": pet_resampled.GetOrigin(),
    "spacing": pet_resampled.GetSpacing(),
    "direction": pet_resampled.GetDirection(),
}
# Arrays are in (Z, Y, X) order.
# So need to modify the order of shape, origin, spacing to (X, Y, Z) for later use.
pet_ct_info["shape"] = pet_ct_info["shape"][::-1]
pet_ct_info["origin"] = pet_ct_info["origin"][::-1]
pet_ct_info["spacing"] = pet_ct_info["spacing"][::-1]

# print all info of pet_ct_info
print("PET resampled info:", pet_ct_info)

if ct_arr.shape != pet_arr.shape:
    raise ValueError(f"Shape mismatch after resample: CT {ct_arr.shape} vs PET {pet_arr.shape}")

- 各向同性处理

In [ ]:
min_spacing = min(pet_ct_info["spacing"])
print(f"Minimum spacing: {min_spacing}")
ct_arr_isotropic = set_isotropy(ct_arr, pet_ct_info["spacing"], new_spacing=min_spacing)
pet_arr_isotropic = set_isotropy(pet_arr, pet_ct_info["spacing"], new_spacing=min_spacing)
print(f"after set isotropy: CT {ct_arr_isotropic.shape} vs PET {pet_arr_isotropic.shape}")

- 可视化
（CT - PET - Fusion）

In [ ]:
slice_index_percentage = 0.50 # change this to select different slices
orientation_index = 1 # 0 for axial, 1 for coronal, 2 for sagittal
alpha = 0.5  # PET weight in fusion

slice_index = int(ct_arr_isotropic.shape[orientation_index] * slice_index_percentage)
if orientation_index == 0:
    ct_slice = ct_arr_isotropic[slice_index, :, :]
    pet_slice = pet_arr_isotropic[slice_index, :, :]
elif orientation_index == 1:
    ct_slice = ct_arr_isotropic[:, slice_index, :]
    pet_slice = pet_arr_isotropic[:, slice_index, :]
elif orientation_index == 2:
    ct_slice = ct_arr_isotropic[:, :, slice_index]
    pet_slice = pet_arr_isotropic[:, :, slice_index]

ct_norm_slice = normalize(ct_slice)
pet_norm_slice = normalize(pet_slice)
fused_slice = (1 - alpha) * ct_norm_slice + alpha * pet_norm_slice

if orientation_index == 1:
    ct_norm_slice = np.fliplr(ct_norm_slice)
    pet_norm_slice = np.fliplr(pet_norm_slice)
    ct_norm_slice = np.rot90(ct_norm_slice, 2)
    pet_norm_slice = np.rot90(pet_norm_slice, 2)

# 如果没有data/data_save文件夹则创建
import os
if not os.path.exists("data/data_save"):
    os.makedirs("data/data_save")

fig, ax = plt.subplots(figsize=(18, 18))
ax.imshow(ct_norm_slice, cmap="gray")
ax.set_title("CT")
ax.axis("off")
fig.savefig("data/data_save/ct_slice.png", bbox_inches="tight", pad_inches=0)
plt.show()

fig, ax = plt.subplots(figsize=(18, 18))
ax.imshow(pet_norm_slice, cmap="gray_r")
ax.set_title("PET")
ax.axis("off")
fig.savefig("data/data_save/pet_slice.png", bbox_inches="tight", pad_inches=0)
plt.show()

fig, ax = plt.subplots(figsize=(18, 18))
ax.imshow(ct_norm_slice, cmap="gray")
ax.imshow(pet_norm_slice, cmap="hot", alpha=alpha)
ax.set_title("Fusion")
ax.axis("off")
fig.savefig("data/data_save/fusion_slice.png", bbox_inches="tight", pad_inches=0)
plt.show()


- only PET in “hot” cmap

In [ ]:
fig, ax = plt.subplots(figsize=(18, 18))
ax.imshow(pet_norm_slice, cmap="hot")
ax.set_title("PET")
ax.axis("off")
fig.savefig("data/data_save/pet_slice_hot.png", bbox_inches="tight", pad_inches=0)
plt.show()

- 正位 MIP 图像

In [ ]:
# PET MIP visualization
orientation_index = 1  # 0 axial, 1 coronal, 2 sagittal

# MIP along the chosen axis (arrays are in Z, Y, X)
pet_mip = np.max(pet_arr_isotropic, axis=orientation_index)
pet_mip_norm = normalize(pet_mip)

# Optional orientation adjustment for coronal view
if orientation_index == 1:
    pet_mip_norm = np.fliplr(pet_mip_norm)
    pet_mip_norm = np.rot90(pet_mip_norm, 2)

fig, ax = plt.subplots(figsize=(18, 18))
ax.imshow(pet_mip_norm, cmap="gray_r")
ax.set_title("PET MIP")
ax.axis("off")
fig.savefig("data/data_save/pet_mip_gray.png", bbox_inches="tight", pad_inches=0)
plt.show()

fig, ax = plt.subplots(figsize=(18, 18))
ax.imshow(pet_mip_norm, cmap="hot")
ax.set_title("PET MIP")
ax.axis("off")
fig.savefig("data/data_save/pet_mip_hot.png", bbox_inches="tight", pad_inches=0)
plt.show()


- 侧位 MIP 图像

In [ ]:
# PET MIP visualization
orientation_index = 2  # 0 axial, 1 coronal, 2 sagittal

# MIP along the chosen axis (arrays are in Z, Y, X)
pet_mip_coronal = np.max(pet_arr_isotropic, axis=orientation_index)
pet_mip_norm_coronal = normalize(pet_mip_coronal)

# Optional orientation adjustment for coronal view
if orientation_index == 2:
    pet_mip_norm_coronal = np.fliplr(pet_mip_norm_coronal)
    pet_mip_norm_coronal = np.rot90(pet_mip_norm_coronal, 2)

plt.figure(figsize=(18, 18))
plt.imshow(pet_mip_norm_coronal, cmap="gray_r")
plt.title("PET MIP")
plt.axis("off")
plt.show()

plt.figure(figsize=(18, 18))
plt.imshow(pet_mip_norm_coronal, cmap="hot")
plt.title("PET MIP")
plt.axis("off")
plt.show()

- CT MIP 图像

In [ ]:
# PET MIP visualization
orientation_index = 1  # 0 axial, 1 coronal, 2 sagittal

# MIP along the chosen axis (arrays are in Z, Y, X)
ct_mip = np.max(ct_arr_isotropic, axis=orientation_index)
ct_mip_norm = normalize(ct_mip)

# Optional orientation adjustment for coronal view
if orientation_index == 1:
    ct_mip_norm = np.fliplr(ct_mip_norm)
    ct_mip_norm = np.rot90(ct_mip_norm, 2)

fig, ax = plt.subplots(figsize=(18, 18))
ax.imshow(ct_mip_norm, cmap="gray_r")
ax.set_title("CT MIP")
ax.axis("off")
# fig.savefig("data/data_save/ct_mip_gray.png", bbox_inches="tight", pad_inches=0)
plt.show()


- CT MIP 与 PET MIP 融合

In [ ]:
fig, ax = plt.subplots(figsize=(18, 18))
ax.imshow(ct_mip_norm, cmap="gray")
ax.imshow(pet_mip_norm, cmap="hot", alpha=0.65)
ax.set_title("Fusion")
ax.axis("off")
plt.show()